In [155]:
# try pytorch pruning

import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim


In [156]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# class LeNet(nn.Module):
#     def __init__(self):
#         super(LeNet, self).__init__()
#         # 1 input image channel, 6 output channels, 3x3 square conv kernel
#         self.conv1 = nn.Conv2d(1, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, int(x.nelement() / x.shape[0]))
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# model = LeNet().to(device=device)

# # Initialize optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [157]:
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# def save_model_as(name, model):
#     torch.save(
#                     {
#                         "model_state_dict": model.state_dict()
#                     },
#                     name
#     )

# def get_file_name(appending_name):
#     return f"model_optim_states_{appending_name}.pth"

# original_model_file_name = get_file_name("original")
# save_model_as(original_model_file_name, model)
# !du -h $original_model_file_name

In [158]:
## model = LeNet()

# parameters_to_prune = (
#     (model.conv1, 'weight'),
#     (model.conv2, 'weight'),
#     (model.fc1, 'weight'),
#     (model.fc2, 'weight'),
#     (model.fc3, 'weight'),
# )

# prune.global_unstructured(
#     parameters_to_prune,
#     pruning_method=prune.L1Unstructured,
#     amount=0.2,
# )


In [159]:
# print(
#     "Sparsity in conv1.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.conv1.weight == 0))
#         / float(model.conv1.weight.nelement())
#     )
# )
# print(
#     "Sparsity in conv2.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.conv2.weight == 0))
#         / float(model.conv2.weight.nelement())
#     )
# )
# print(
#     "Sparsity in fc1.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc1.weight == 0))
#         / float(model.fc1.weight.nelement())
#     )
# )
# print(
#     "Sparsity in fc2.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc2.weight == 0))
#         / float(model.fc2.weight.nelement())
#     )
# )
# print(
#     "Sparsity in fc3.weight: {:.2f}%".format(
#         100. * float(torch.sum(model.fc3.weight == 0))
#         / float(model.fc3.weight.nelement())
#     )
# )
# print(
#     "Global sparsity: {:.2f}%".format(
#         100. * float(
#             torch.sum(model.conv1.weight == 0)
#             + torch.sum(model.conv2.weight == 0)
#             + torch.sum(model.fc1.weight == 0)
#             + torch.sum(model.fc2.weight == 0)
#             + torch.sum(model.fc3.weight == 0)
#         )
#         / float(
#             model.conv1.weight.nelement()
#             + model.conv2.weight.nelement()
#             + model.fc1.weight.nelement()
#             + model.fc2.weight.nelement()
#             + model.fc3.weight.nelement()
#         )
#     )
# )

In [160]:
# pruned_filed_name = get_file_name("pruned")
# save_model_as(pruned_filed_name, model)
# !du -h $pruned_filed_name

In [161]:
# import torch.quantization

# quantized_model = torch.quantization.quantize_dynamic(
#     model, {torch.nn.Linear}, dtype=torch.qint8
# )
# # model, {torch.nn.Linear}, dtype=torch.float16

# pruned_filed_name = get_file_name("quantized")
# save_model_as(pruned_filed_name, quantized_model)
# !du -h $pruned_filed_name


In [162]:
# load the model
import os
check_point_folder = "/Users/leiwang/repos/DenseDepth-Pytorch/checkpoints/"
check_point_path = os.path.join(check_point_folder, "ckpt_0_29.pth")
loaded_model = torch.load(check_point_path, map_location='cpu')


In [163]:
import os
import argparse as arg
import shutil

import torch

import numpy as np
# import cv2
from PIL import Image
from glob import glob

from data import load_testloader
from model import DenseDepth
from utils import colorize, DepthNorm, AverageMeter, load_images
from losses import ssim as ssim_criterion
from losses import depth_loss as gradient_criterion

# load check point and populate model class
model = DenseDepth(encoder_pretrained=False)
device = 'cpu'
ckpt = torch.load(check_point_path, map_location=torch.device(device))
model.load_state_dict(ckpt["model_state_dict"])
model = model.to(device)

# save model only
check_point_model_only_path = os.path.join(check_point_folder, "ckpt_0_29_model.pth")
torch.save(model.state_dict(), check_point_model_only_path)
    

In [164]:
# quantize it and save it

import torch.quantization

# quantize it

# A. dynamic version
# quantized_model = torch.quantization.quantize_dynamic(
#     model, {torch.nn.Linear}, dtype=torch.qint8
# )
# quantized_model.eval()

# B. static version
backend = "fbgemm"#"qnnpack"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
quantized_model = torch.quantization.prepare(model, inplace=False)
quantized_model = torch.quantization.convert(quantized_model.eval(), inplace=False)
quantized_model.eval()

# save the model only
check_point_model_only_quanited_path = os.path.join(check_point_folder, "ckpt_0_29_model_quantized.pth")
torch.save(quantized_model.state_dict(), check_point_model_only_quanited_path)

# save the whole check point
quantized_full_object = ckpt
quantized_full_object['model_state_dict'] = quantized_model.state_dict()

check_point_path_quantized = os.path.join(check_point_folder, "ckpt_0_29_quantized.pth")
torch.save(quantized_full_object, check_point_path_quantized)

In [165]:
# print(ckpt["model_state_dict"].keys())

In [166]:
# print(quantized_model.state_dict().keys())

In [167]:
# !python /Users/leiwang/repos/DenseDepth-Pytorch/densedepth/test.py --checkpoint $check_point_path \
#                                               --device "cpu" \
#                                               --data "/Users/leiwang/repos/DenseDepth-Pytorch/examples/raw/" \
#                                               --output_path "/Users/leiwang/repos/DenseDepth-Pytorch/examples/postprocessed/" \
#                                               --cmap "twilight"


In [168]:
!python /Users/leiwang/repos/DenseDepth-Pytorch/densedepth/test.py --checkpoint $check_point_path_quantized \
                                              --device "cpu" \
                                              --data "/Users/leiwang/repos/DenseDepth-Pytorch/examples/raw/" \
                                              --output_path "/Users/leiwang/repos/DenseDepth-Pytorch/examples/postprocessed_quantized/" \
                                              --cmap "twilight"


Using torch version:  1.10.0
Using device: cpu
Traceback (most recent call last):
  File "/Users/leiwang/repos/DenseDepth-Pytorch/densedepth/test.py", line 103, in <module>
    main()
  File "/Users/leiwang/repos/DenseDepth-Pytorch/densedepth/test.py", line 48, in main
    model.load_state_dict(ckpt["model_state_dict"])
  File "/opt/miniconda3/envs/DenseDepth-Pytorch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1482, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for DenseDepth:
	Missing key(s) in state_dict: "encoder.densenet.classifier.weight", "encoder.densenet.classifier.bias". 
	Unexpected key(s) in state_dict: "encoder.densenet.features.conv0.bias", "encoder.densenet.features.conv0.scale", "encoder.densenet.features.conv0.zero_point", "encoder.densenet.features.denseblock1.denselayer1.conv1.bias", "encoder.densenet.features.denseblock1.denselayer1.conv1.scale", "encoder